In [ ]:
from IPython.display import display, HTML, Image

In [ ]:
!git clone https://github.com/KU-DIC/LG_time_series_day05.git #코랩 사용

# [머신러닝 기반 시계열 분석 2 실습]
# ANN
## [Deep Neural Networks - Regression(수치 예측)]

##### jupyter notebook 단축키

- ctrl+enter: 셀 실행   
- shift+enter: 셀 실행 및 다음 셀 이동   
- alt+enter: 셀 실행, 다음 셀 이동, 새로운 셀 생성
- a: 상단에 새로운 셀 만들기
- b: 하단에 새로운 셀 만들기
- dd: 셀 삭제(x: 셀 삭제)
- 함수 ( ) 안에서 shift+tab: arguments description. shift+tab+tab은 길게 볼 수 있도록

## 1. 모듈 불러오기

In [ ]:
''' 기본 모듈 및 시각화 모듈 '''
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

''' 데이터 전처리 모듈 '''
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

'''Neural Network을 위한 딥러닝 모듈'''
import torch

''' 기타 optional'''
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

## 2. 분석데이터 : bike_train

### 데이터 불러오기

In [ ]:
data = pd.read_csv('/content/LG_time_series_day05/data/bike_train.csv')
# data = pd.read_csv('./data/bike_train.csv') #로컬

## 2.1 데이터 전처리 및 탐색적 데이터 분석

### 데이터 확인

In [ ]:
print('Data shape: {}'.format(data.shape))
data.head()

In [ ]:
data.info()

<li>datatime : 날짜</li>
<li>season : 1 = 봄, 2 = 여름, 3 = 가을, 4 = 겨울</li>
<li>holiday : 1 = 주말 및 휴일, 0 = 평일</li>
<li>workingday: 1 = 주중, 0 = 주말 및 휴일</li>
<li>weather : 1 = 맑음, 약간 구름 낀 흐림, 2 = 안개, 안개 + 흐림, 3 = 가벼운 눈, 가벼운 비 + 천둥, 4 = 심한 눈/비, 천둥/번개</li>
<li>temp : 온도(섭씨)</li>
<li>atem : 체감 온도(섭씨)</li>
<li>humidity : 상대 습도</li>
<li>windspeed : 풍속</li>
<li>casual : 미등록 사용자 대여수</li>
<li>registered : 등록된 사용자 대여수</li>
<li>count : 대여 횟수(casual + registered) ==> [Target variable]</li>

### 데이터 전처리

In [ ]:
drop_columns = ['datetime','casual','registered'] # 날짜, 미등록 사용자 대여수, 등록된 사용자 대여수 제거
data.drop(drop_columns, axis=1,inplace=True)
data.head()

- 'datetime' 칼럼은 시간 정보를 포함하고 있지만 여기서는 고려하지 않음

- 'casual'과 'registered' 이 두 feature를 더하면 label data인 count이기 때문에, 학습할 때 이를 포함하는 것은 오히려 학습 모델이 overfitting 되도록 하므로 고려하지 않음

### 범주형 설명변수에 대한 Dummy 변수 생성

In [ ]:
data_continuous = data.drop(labels=['season', 'holiday', 'workingday', 'weather'], axis=1) 

data_season_dummy = pd.get_dummies(data=data['season'], prefix='season', drop_first=True)
data_holiday_dummy = pd.get_dummies(data=data['holiday'], prefix='holiday', drop_first=True)
data_workingday_dummy = pd.get_dummies(data=data['workingday'], prefix='workingday', drop_first=True)
data_weather_dummy = pd.get_dummies(data=data['weather'], prefix='weather', drop_first=True)

data_dummied = pd.concat(objs=[data_continuous, data_season_dummy, data_holiday_dummy, data_workingday_dummy, data_weather_dummy], axis=1)

data_dummied.head()

- 'season', 'holiday', 'workingday', 'weather' feature는 모두 category variable이기 때문에,
- 범주형 설명변수에 대해 get_dummies를 사용해 더비 변수를 생성하고, 새로운 데이터셋을 생성

### 설명변수(X)와 반응변수(Y) 정의

In [ ]:
y = data_dummied['count']
x = data_dummied.drop(['count'], axis=1, inplace=False)

### 학습데이터(Training Dataset)와 테스트 데이터(Testing Dataset) 분리

In [ ]:
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.3)

### 정규화 : Standardization(표준정규화) with Standard Scaler

In [ ]:
x.describe()

In [ ]:
scaler = StandardScaler()
scaler.fit(train_x)

train_x = scaler.transform(train_x)
test_x = scaler.transform(test_x)

In [ ]:
train_x

### Tensor 구조 활용
- pytorch로 학습을 하기 위해서는, data의 구조가 tensor의 형태가 되어야 함
- 따라서 Dataframe, Series 구조로 구성된 x, y를 tensor구조로 변환해줘야 함

In [ ]:
torch.from_numpy(np.array(train_y)).float()

In [ ]:
torch.as_tensor(train_x.astype('float'))

In [ ]:
def df_to_tensor(df):
    return torch.from_numpy(np.array(df)).float()

train_x = df_to_tensor(train_x)
test_x = df_to_tensor(test_x)
train_y = df_to_tensor(train_y)
test_y = df_to_tensor(test_y)

print("train_x : ", '\n', train_x, '\n\n',
      "test_x : ", '\n', test_x, '\n\n',
      "train_y : ", train_y, '\n\n',
      "test_y : ", test_y)

## 2.2 PyTorch를 이용한 DNN 구조

- nn.Linear: nn.Module의 subclass로 in_features개의 input을 선형변환을 거쳐 out_features개의 output으로 변환
- 1개의 input layer, 3개의 hidden layer, 1개의 output layer로 구성되어 총 5개의 layer로 구성된 DNN 모델
- 계산의 과정 중간에 쓰이는 activation function으로는 ReLU 활성화 함수에서 발전된 함수인 LeakyReLU 활성화 함수를 사용

In [ ]:
class NeuralNet(torch.nn.Module):
    def __init__(self, input_size, hidden_size_1, hidden_size_2, hidden_size_3):
        super(NeuralNet, self).__init__()
        self.input_size = input_size
        
        self.hidden_size_1 = hidden_size_1
        self.hidden_size_2 = hidden_size_2
        self.hidden_size_3 = hidden_size_3
        
        self.linear_1 = torch.nn.Linear(self.input_size, self.hidden_size_1)
        self.linear_2 = torch.nn.Linear(self.hidden_size_1, self.hidden_size_2)
        self.linear_3 = torch.nn.Linear(self.hidden_size_2, self.hidden_size_3)
        self.linear_4 = torch.nn.Linear(self.hidden_size_3, 1)
        
        self.LeakyReLU = torch.nn.LeakyReLU()
    
    
    def forward(self, input_tensor):
        linear1 = self.linear_1(input_tensor)
        LeakyReLU1 = self.LeakyReLU(linear1)
        linear2 = self.linear_2(LeakyReLU1)
        LeakyReLU2 = self.LeakyReLU(linear2)
        linear3 = self.linear_3(LeakyReLU2)
        LeakyReLU3 = self.LeakyReLU(linear3)
        linear4 = self.linear_4(LeakyReLU3)
        output = linear4 
        #output layer 단계에서는 회귀 값 그대로를 사용해야 하기 때문에, 별다른 활성화함수를 사용하지 않고, Linear 활성화 함수를 사용하거나 아예 사용하지 않음
        return output

<center><a href=https://machinelearningmastery.com/choose-an-activation-function-for-deep-learning/'> https://machinelearningmastery.com/choose-an-activation-function-for-deep-learning/ </a></center>

### 모델 및 비용함수, Solver 설정

- input layer의 개수는 설명변수의 개수인 12개에서 시작해서, 144, 64, 8로 계산하고 마지막에는 1개의 output을 얻는 과정을 진행한다.
- loss function과 optimizer를 결정하는 것은 학습 속도와 성능을 결정짓는 중요한 부분

(이 예제에서는 loss function으로 L1Loss함수를 사용하고, optimizer는 AdamOptimizer를 사용).

In [ ]:
model = NeuralNet(12, 144, 64, 8)
# 비용함수 정의
criterion = torch.nn.L1Loss()

# 경사하강법의 종류 정의 (adam)
solver = torch.optim.Adam(model.parameters(), lr=0.001)

### DNN Regressor 학습

### Train 과정

- 모델에 데이터를 통과시켜 예측값(현재 모델의 weights로 prediction)을 계산
- 실제 정답과 예측값의 loss를 비교
- Gradient를 계산
- 이 값을 통해 weights를 업데이트 (backpropagation)

1. model.train(): 모델을 학습모드로 변경
2. solver.zero_grad(): optimizer의 grad를 0으로 설정
    PyTorch는 parameter들의 gradient를 계산해줄 때 grad는 계속 누적되도록 되어 있음
    따라서 새로운 epoch이 시작될 때, gradient를 다시 계산할 때에는 0으로 세팅해주어야 함
3. train_output: 모델에 학습데이터(X_train)을 넣었을 때 예측값
4. train_loss: criterion이 L1Loss로 설정되어 있으므로, train_output과 y_train의 평균절대오차(MAE)를 계산
4. train_loss.backward(): gradient 계산을 역전파(backpropagation)로 진행
5. optimizer.step(): 계산한 gradient를 토대로 parameter를 업데이트

In [ ]:
epochs=1000

print("Start Training !")
print('-'*50)

train_loss_total = []

for epoch in range(epochs):

    model.train()
    solver.zero_grad()
    train_output = model(train_x)
    train_loss = criterion(train_output.squeeze(), train_y)
    if epoch % 100 == 0:
        print('[%d epoch] Train loss : %.3f' % (epoch+1, train_loss.item()))
    train_loss.backward()
    solver.step()
       
    best_model = model.state_dict()
    
    train_loss_total.append(train_loss.item())

### 학습 상태 확인 (learning curve)

In [ ]:
plt.figure(figsize=(20,10))

# 학습 및 검증 로스 변동 관찰하기
plt.plot(train_loss_total,label='Train Loss')

plt.legend(fontsize=20)
plt.title("Learning Curve of trained DNN Regressor", fontsize=18)
plt.show()

## 2-3. DNN 모델 성능 평가

### 학습된 DNN Regressor 결과 확인 및 성능 평가 : Training Data

In [ ]:
model.eval()
test_loss = criterion(torch.squeeze(model(test_x)), test_y)
print('After Training, test loss is {}'.format(test_loss.item()))